In [ ]:
"""
Module pour charger les données depuis BigQuery
"""
from google.cloud import bigquery
import pandas as pd
from typing import Optional, Tuple
import os
from dotenv import load_dotenv

load_dotenv()

class BigQueryLoader:
    """Classe pour gérer les requêtes BigQuery"""
    
    def __init__(self, project_id: Optional[str] = None):
        self.project_id = project_id or os.getenv('GCP_PROJECT')
        self.client = bigquery.Client(project=self.project_id)
        
    def get_movies(self, limit: Optional[int] = None) -> pd.DataFrame:
        """Charger la table movies"""
        query = """
        SELECT *
        FROM `master-ai-cloud.MoviePlatform.movies`
        """
        if limit:
            query += f" LIMIT {limit}"
            
        return self.client.query(query).to_dataframe()
    
    def get_ratings(self, limit: Optional[int] = None) -> pd.DataFrame:
        """Charger la table ratings"""
        query = """
        SELECT userId, movieId, rating, timestamp
        FROM `master-ai-cloud.MoviePlatform.ratings`
        """
        if limit:
            query += f" LIMIT {limit}"
            
        return self.client.query(query).to_dataframe()
    
    def get_user_ratings(self, user_id: int) -> pd.DataFrame:
        """Récupérer les ratings d'un utilisateur spécifique"""
        query = f"""
        SELECT r.userId, r.movieId, r.rating, m.title, m.genres
        FROM `master-ai-cloud.MoviePlatform.ratings` r
        JOIN `master-ai-cloud.MoviePlatform.movies` m
        ON r.movieId = m.movieId
        WHERE r.userId = {user_id}
        ORDER BY r.timestamp DESC
        """
        return self.client.query(query).to_dataframe()
    
    def get_movie_info(self, movie_id: int) -> pd.DataFrame:
        """Récupérer les infos d'un film"""
        query = f"""
        SELECT *
        FROM `master-ai-cloud.MoviePlatform.movies`
        WHERE movieId = {movie_id}
        """
        return self.client.query(query).to_dataframe()
    
    def get_sample_data(self, n_users: int = 1000) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Charger un échantillon de données pour le développement
        
        Args:
            n_users: Nombre d'utilisateurs à inclure
            
        Returns:
            Tuple de (ratings_df, movies_df)
        """
        # Sélectionner des utilisateurs avec suffisamment de ratings
        query_users = f"""
        SELECT userId
        FROM `master-ai-cloud.MoviePlatform.ratings`
        GROUP BY userId
        HAVING COUNT(*) >= 20
        ORDER BY RAND()
        LIMIT {n_users}
        """
        
        users_df = self.client.query(query_users).to_dataframe()
        user_ids = users_df['userId'].tolist()
        user_ids_str = ','.join(map(str, user_ids))
        
        # Charger les ratings de ces utilisateurs
        query_ratings = f"""
        SELECT *
        FROM `master-ai-cloud.MoviePlatform.ratings`
        WHERE userId IN ({user_ids_str})
        """
        
        ratings_df = self.client.query(query_ratings).to_dataframe()
        
        # Charger les films correspondants
        movie_ids = ratings_df['movieId'].unique().tolist()
        movie_ids_str = ','.join(map(str, movie_ids))
        
        query_movies = f"""
        SELECT *
        FROM `master-ai-cloud.MoviePlatform.movies`
        WHERE movieId IN ({movie_ids_str})
        """
        
        movies_df = self.client.query(query_movies).to_dataframe()
        
        return ratings_df, movies_df
    
    def save_to_storage(self, df: pd.DataFrame, table_name: str):
        """Sauvegarder un DataFrame dans BigQuery dans votre dataset"""
        dataset_id = os.getenv('DATASET_ID')
        table_id = f"{self.project_id}.{dataset_id}.{table_name}"
        
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_TRUNCATE",
        )
        
        job = self.client.load_table_from_dataframe(
            df, table_id, job_config=job_config
        )
        job.result()
        
        print(f"Table {table_id} créée/mise à jour avec {len(df)} lignes")


